In [1]:
!pip install dlt
!pip install minio
!pip install --upgrade pyarrow
!pip install --upgrade dlt
!pip install "dlt[s3]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.9 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 4.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.0/518.0 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 

In [2]:
!pip install boto3

  Using cached botocore-1.40.36-py3-none-any.whl.metadata (5.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 242.7 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 291.6 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.40.18
    Uninstalling botocore-1.40.18:
      Successfully uninstalled botocore-1.40.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.24.2 requires botocore<1.40.19,>=1.40.15, but you have botocore 1.40.36 which is incompatible.


In [4]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 3.0 MB/s eta 0:00:0000:0100:01


In [5]:
import dlt
import pyarrow.parquet as pq
import requests
import pandas as pd
import os
import tempfile
import pyarrow.parquet as pq
from minio import Minio

In [6]:
import requests
import tempfile
import pyarrow.parquet as pq
import dlt
import boto3
import os

# Configuración de la URL
URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet"

@dlt.resource(name="yellow_trip_data", write_disposition="replace")
def yellow_trip_data():
    try:
        print("Descargando archivo parquet...")
        response = requests.get(URL, stream=True)
        response.raise_for_status()

        with tempfile.NamedTemporaryFile(suffix=".parquet") as tmp:
            tmp.write(response.content)
            tmp.flush()

            # Leer en formato Arrow
            table = pq.read_table(tmp.name)
            print(f"Datos cargados en Arrow: {table.num_rows} filas, {table.num_columns} columnas")
            yield table

    except Exception as e:
        print(f"Error al descargar o procesar datos: {e}")
        raise


def upload_to_minio(local_folder, bucket_name="my-bucket"):
    """Sube todos los archivos generados a MinIO"""
    s3 = boto3.client(
        "s3",
        endpoint_url="http://minio:9000",
        aws_access_key_id="admin",
        aws_secret_access_key="password",
    )
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            local_path = os.path.join(root, file)
            s3_key = os.path.relpath(local_path, local_folder)
            print(f"Subiendo {local_path} a s3://{bucket_name}/{s3_key}")
            s3.upload_file(local_path, bucket_name, s3_key)


def main():
    try:
        # Configurar pipeline
        pipeline = dlt.pipeline(
            pipeline_name="yellow_trip_data_pipeline",
            destination="filesystem",  # Guarda primero en local
            dataset_name="yellow_trip",
            dev_mode=False,
        )
        
        # Ejecutar el pipeline
        print("Iniciando ingesta de datos...")
        load_info = pipeline.run(
            yellow_trip_data(),
            loader_file_format="parquet",
        )
        
        print("Pipeline ejecutado exitosamente!")
        print(f"Load info: {load_info}")

        # Subir resultados al bucket bronze en MinIO
        output_path = "/workspace/output"
        upload_to_minio(output_path, "bronze")
        print("Datos subidos a MinIO satisfactoriamente.")

    except Exception as e:
        print(f"Error en el pipeline: {e}")
        raise


if __name__ == "__main__":
    main()


Iniciando ingesta de datos...
Error en el pipeline: Pipeline execution failed at `step=sync` with exception:

<class 'dlt.common.configuration.exceptions.ConfigFieldMissingException'>
Missing fields in configuration: ['bucket_url'] FilesystemDestinationClientConfiguration
	for field `bucket_url` the following (config providers, keys) were tried in order:
		(Environment Variables, YELLOW_TRIP_DATA_PIPELINE__DESTINATION__FILESYSTEM__BUCKET_URL)
		(Environment Variables, YELLOW_TRIP_DATA_PIPELINE__DESTINATION__BUCKET_URL)
		(Environment Variables, YELLOW_TRIP_DATA_PIPELINE__BUCKET_URL)
		(Environment Variables, DESTINATION__FILESYSTEM__BUCKET_URL)
		(Environment Variables, DESTINATION__BUCKET_URL)
		(Environment Variables, BUCKET_URL)

Provider `secrets.toml` loaded values from locations:
	- /workspace/.dlt/secrets.toml
	- /home/jovyan/.dlt/secrets.toml
Provider `config.toml` loaded values from locations:
	- /workspace/.dlt/config.toml
	- /home/jovyan/.dlt/config.toml

If you keep `.dlt` 

PipelineStepFailed: Pipeline execution failed at `step=sync` with exception:

<class 'dlt.common.configuration.exceptions.ConfigFieldMissingException'>
Missing fields in configuration: ['bucket_url'] FilesystemDestinationClientConfiguration
	for field `bucket_url` the following (config providers, keys) were tried in order:
		(Environment Variables, YELLOW_TRIP_DATA_PIPELINE__DESTINATION__FILESYSTEM__BUCKET_URL)
		(Environment Variables, YELLOW_TRIP_DATA_PIPELINE__DESTINATION__BUCKET_URL)
		(Environment Variables, YELLOW_TRIP_DATA_PIPELINE__BUCKET_URL)
		(Environment Variables, DESTINATION__FILESYSTEM__BUCKET_URL)
		(Environment Variables, DESTINATION__BUCKET_URL)
		(Environment Variables, BUCKET_URL)

Provider `secrets.toml` loaded values from locations:
	- /workspace/.dlt/secrets.toml
	- /home/jovyan/.dlt/secrets.toml
WARNING: provider `secrets.toml` is empty. Locations (i.e., files) are missing or empty.
Provider `config.toml` loaded values from locations:
	- /workspace/.dlt/config.t

In [ ]:
!cat .dlt/secrets.toml